In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input

In [2]:
# Load and preprocess the CIFAR-10 dataset
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
train_images = train_images.astype('float32') / 255
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

170498071/170498071 [==============================] - 4s 0us/step


In [3]:
# Create a tf.data.Dataset for training and testing
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))

In [4]:
# Define preprocessing function
def preprocess_image(image, label):
    image = tf.image.resize(image, (224, 224))
    image = preprocess_input(image)
    return image, label

In [5]:
# Apply preprocessing function to the datasets
train_dataset = train_dataset.map(preprocess_image)
test_dataset = test_dataset.map(preprocess_image)

In [6]:
# Batch and prefetch the datasets
batch_size = 32
train_dataset = train_dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

In [7]:
# Load pre-trained InceptionV3 model without top (fully connected layers)
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

87910968/87910968 [==============================] - 1s 0us/step


In [8]:
# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

In [9]:
# Build your model on top of the pre-trained model
model = models.Sequential()
model.add(base_model)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(10, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
# Train the model using the tf.data.Dataset
model.fit(train_dataset, epochs=3, validation_data=test_dataset)

Epoch 1/3
1563/1563 [==============================] - 237s 145ms/step - loss: 2.2028 - accuracy: 0.1891 - val_loss: 2.1085 - val_accuracy: 0.2543
Epoch 2/3
1563/1563 [==============================] - 228s 146ms/step - loss: 2.0518 - accuracy: 0.2720 - val_loss: 2.0010 - val_accuracy: 0.3069
Epoch 3/3
1563/1563 [==============================] - 229s 147ms/step - loss: 1.9644 - accuracy: 0.3109 - val_loss: 1.9302 - val_accuracy: 0.3360


In [11]:
# Evaluate the model on the test set
evaluation_results = model.evaluate(test_dataset)

# Extract loss and accuracy from evaluation results
loss, accuracy = evaluation_results

# Print the evaluation results
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy * 100:.2f}%")

313/313 [==============================] - 38s 121ms/step - loss: 1.9302 - accuracy: 0.3360
Test Loss: 1.9302
Test Accuracy: 33.60%
